In [1]:
!pip install img2dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 12.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=9e447d45626565bfda5d32af9b92ff67575b9f3883ce0c2f62edbf3ad3efcb76
  Stored in directory: /root/.cache/pip/wheels/26/9a/dd/2818b1b023daf077ec3e625c47ae446aca587a5abe48e05212
Successfully built fire
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: fss

In [2]:
path_json = '../input/guie-laion5b-dataset/GUIE_laion5b_dataset_en.json'
!ls $path_json

../input/guie-laion5b-dataset/GUIE_laion5b_dataset_en.json


In [3]:
!pip install wandb


In [4]:
from typing import Dict, List, Callable
from pathlib import Path
from PIL import Image

import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch.optim as optim
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
import numpy as np
import wandb
from pytorch_lightning.loggers.wandb import WandbLogger


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
wandb.login()  # To confirm login

wandb: Currently logged in as: ashllxyy. Use `wandb login --relogin` to force relogin


True

In [7]:
#DataConfig
num_pairs_taken = 20_000 # Number of pairs to download/build vocabulary on
max_seq_length = 75 # Maximum sequence length for tokenized sentences

In [8]:
df = pd.read_json(path_json)
df['text'] = df['caption']
df['text_en'] = df['caption_en']

df.drop(columns=['caption', 'caption_en'], inplace=True)

df_head = df.head(num_pairs_taken)
df_head.to_json('./fixed_df.json', orient='records')

In [9]:
out_dir = Path('./dataset')
out_dir.mkdir(exist_ok=True)
!img2dataset "fixed_df.json" --input_format="json" --caption_col="text_en" --output_folder=$out_dir --processes_count=1 --output_format="files" --resize_mode="no"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Starting the downloading of this file
Sharding file number 1 of 1 called /kaggle/working/fixed_df.json
0it [00:00, ?it/s]File sharded in 2 shards
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
1it [04:41, 281.27s/it]worker  - success: 0.872 - failed to download: 0.115 - failed to resize: 0.014 - 

In [10]:
df = pd.read_json('/kaggle/input/guie-laion5b-dataset/GUIE_laion5b_dataset_en.json')
df = df.loc[:, ['url', 'caption_en']]
df = df.iloc[:num_pairs_taken]

# Building vocabulary

PAD_token = 0   # Used for padding short sentences
SOS_token = 1   # Start-of-sentence token
EOS_token = 2   # End-of-sentence token
CLS_token = 3   # Classification token

class Vocabulary():
    def __init__(self, name):
      self.name = name
      self.word2index = {}
      self.word2count = {}
      self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", CLS_token: "CLS"}
      self.num_words = 3
      self.num_sentences = 0
      self.longest_sentence = 0
    
    def add_word(self, word):
      if word not in self.word2index:
        # First entry of word into vocabulary
        self.word2index[word] = self.num_words
        self.word2count[word] = 1
        self.index2word[self.num_words] = word
        self.num_words += 1
      else:
        # Word exists; increase word count
        self.word2count[word] += 1
    
    def add_sentence(self, sentence):
      sentence_len = 0
      for word in sentence.split(' '):
        sentence_len += 1
        self.add_word(word)
        if sentence_len > self.longest_sentence:
        # This is the longest sentence
            self.longest_sentence = sentence_len
      # Count the number of sentences
        self.num_sentences += 1
    
    def to_word(self, index):
      return self.index2word[index]

    def to_index(self, word):
      return self.word2index[word]

clipVocab = Vocabulary('CLIP')


for ind in df.index:
    clipVocab.add_sentence(df['caption_en'][ind])
    
del df
print(f'{clipVocab.num_words} words obtained in Vocabulary')

36443 words obtained in Vocabulary


In [11]:
def get_image_files_dict(base_path: Path) -> Dict:
    image_files = [
        *base_path.glob("**/*.png"),
        *base_path.glob("**/*.jpg"),
        *base_path.glob("**/*.jpeg"),
        *base_path.glob("**/*.bmp"),
    ]
    return {image_file.stem: image_file for image_file in image_files}


def get_text_files_dict(base_path: Path) -> Dict:
    text_files = [*base_path.glob("**/*.txt")]
    return {text_file.stem: text_file for text_file in text_files}


def get_shared_stems(image_files_dict: Dict, text_files_dict: Dict) -> List:
    image_files_stems = set(image_files_dict.keys())
    text_files_stems = set(text_files_dict.keys())
    return list(image_files_stems & text_files_stems)


class TextImageDataset(Dataset):
    """Dataset for text-image pairs"""

    def __init__(
            self,
            root: str,
            preprocess: Callable = None,
            tokenizer: Callable = None,
    ):
        super().__init__()
        self.root = Path(root)
        self.preprocess = preprocess
        self.tokenizer = tokenizer

        self.image_files_dict = get_image_files_dict(self.root)
        self.text_files_dict = get_text_files_dict(self.root)
        self.shared_stems = get_shared_stems(self.image_files_dict, self.text_files_dict)

    def __len__(self):
        return len(self.shared_stems)

    def get_caption(self, text_file: Path):
        with open(text_file, 'r') as f:
            return f.read().strip()

    def __getitem__(self, i: int):
        stem = self.shared_stems[i]
        # read image
        image_file = self.image_files_dict[stem]
        image = Image.open(image_file).convert("RGB")

        # read text
        text_file = self.text_files_dict[stem]
        text = self.get_caption(text_file)

        # preprocess image and text
        if self.preprocess:
            image = self.preprocess(image)
        
        if self.tokenizer:
            text = self.tokenizer(text)
        return image, text


In [12]:
def make_patches(image_inp, num_of_patches):
    n, c, h, w = image_inp.shape

    patch_size_h = h//num_of_patches    
    patch_size_w = w//num_of_patches    

    patches = torch.zeros(n, num_of_patches**2, h * w * c // num_of_patches ** 2) 
    for idx, each_img in enumerate(image_inp):
        for i in range(num_of_patches):
            for j in range(num_of_patches):
                patch = each_img[:, i*patch_size_h: (i+1)*patch_size_h, j*patch_size_w:(j+1)*patch_size_w]  
                patches[idx, i*num_of_patches + j] = patch.flatten()

    return patches

def get_positional_embeddings(sequence_length, depth):
    out = torch.ones(sequence_length, depth)

    for i in range(sequence_length):
        for j in range(depth):
            if j % 2 == 0:
                out[i][j] = np.sin(i / (10_000 ** (j / depth)))
            else:
                out[i][j] = np.cos(i / (10_000 ** ((j - 1) / depth)))
    return out
    

In [75]:
class SelfAttention(pl.LightningModule):
    
    def __init__(self, num_heads, embed_size):
        """
        Multi-Head Attention Layer that separates the embeddings into (num_heads) Heads and computes QKV attention
        
        Arguments:-
        num_heads: Number of attention heads
        embed_size: The input embedding dimension
        """
        super().__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size 
        self.head_dim = embed_size // num_heads 
        
        assert (self.head_dim * num_heads == embed_size), 'Embed size needs to be divisible by number of heads'
        
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
        self.fc_out = nn.Linear(self.head_dim * num_heads, embed_size)

    def forward(self, values, keys, query, mask):
        """
        Forward pass for our Multi-Head Attention layer
        
        Arguments:-
        values: 'Same as below'
        keys: 'Same as below'
        query: (batch_dim, seq_length, seq_dimension) The inputted query matrix.
        """
        N = values.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        
        values = values.reshape(N, value_len, self.num_heads, self.head_dim) 
        queries = query.reshape(N, query_len, self.num_heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.num_heads, self.head_dim)

        values = self.values(values)
        keys = self.values(keys)
        query = self.queries(queries)
        
        values = values.transpose(1,2).contiguous().view(N*self.num_heads, value_len, self.head_dim)
        keys = keys.transpose(1,2).contiguous().view(N*self.num_heads, key_len, self.head_dim) 
        queries = query.transpose(1,2).contiguous().view(N*self.num_heads, query_len, self.head_dim)
        
        energy = torch.bmm(queries, keys.transpose(1, 2)) 
        energy_scaled = energy / (self.embed_size ** 0.5)
        
        attention = F.softmax(energy_scaled, dim=2)
        out = torch.bmm(attention, values).view(N, self.num_heads, query_len, self.head_dim)
        out = out.transpose(1, 2).contiguous().view(N, query_len, self.head_dim * self.num_heads)
        
        return self.fc_out(out)
    
class TransformerBlock(pl.LightningModule):
    
    def __init__(self, embed_dim, dropout, num_heads, forward_expansion):
        """
        The Transformer Block is initialized with a few pre-defined parameters.
        
        Arguments:-
        embed_dim: The embedding dimension for each token
        dropout: The dropout rate
        num_heads: Number of heads to create for Multi-Head Attention layer
        forward_expansion: Arbitrary expansion dimension for the final feed-forward layer, post-attention (Check forward())
        """
        super().__init__()
        self.attention = SelfAttention(num_heads, embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(embed_dim)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * forward_expansion),
            nn.ReLU(),
            nn.Linear(embed_dim * forward_expansion, embed_dim)
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, value, key, query, mask):
        """
        Forward pass for the Transformer
        
        Arguments:-
        value, key, query: (batch_dim, seq_length, seq_dimension) Inputted sequence, Usually the same
        mask: Mask to fill in large -ve values so that the softmax in SelfAttention considers them as 0.
        """
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.layer_norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.layer_norm2(forward + x))
        
        return out
    
class Encoder(pl.LightningModule):
    
    def __init__(
        self,
        src_vocab_size,
        embed_dim,
        num_layers,
        num_heads,
        forward_expansion,
        dropout,
        max_length,
        final_output_embed
    ):
        """
        The Encoder block is where the embeddings are formed, passed in through the Transformer Block for (num_layers) times, and
        projected into a final embedding space
        
        Arguments:-
        src_vocab_size: Total number of words in our Source Vocabulary
        embed_dim: Words are embedded into embed_size
        num_layers: Number of times to repeat transformer block
        num_heads: Number of heads to create for Multi-Head Attention layer
        #device: Deprecated for Lightning Implementation since lightning doesn't like device calls :D
        forward_expansion: Arbitrary expansion dimension (Check Transformer Block)
        dropout: Dropout rate
        max_length: Maximum allowed sequence length, aids in getting positional embeddings
        final_output_embed: Final embedding dimension after passing through Transformer Block
        """
        super().__init__()
        self.embed_dim = embed_dim
        self.final_output_embed = final_output_embed
        
        self.word_embedding = nn.Embedding(src_vocab_size, embed_dim)
        self.positional_embedding = nn.Embedding(max_length, embed_dim)
        
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_dim,
                    dropout,
                    num_heads,
                    forward_expansion
                )
                for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, self.final_output_embed)    
        )
    
    def forward(self, inp, mask):
        """
        Forward pass for the Encoder
        
        Arguments:-
        inp: (num_samples, num_seq_length) Input for our Encoder
        mask: The mask to pass to get masked attention
        """
        

        num_samples, seq_length = inp.shape
        positions = torch.arange(0, seq_length).expand(num_samples, seq_length).to(device=self.device)
        word_embed = self.word_embedding(inp)

        pos_embed = self.positional_embedding(positions)
        out = word_embed + pos_embed
        
        out = self.dropout(out)
        for layer in self.layers:
            out = layer(out, out, out, mask) 
            
        out = self.mlp(out[:, 0]) # Taking the (final_text_embed) dimension CLS token
        return out
    
class Transformer(pl.LightningModule):
    
    def __init__(
        self,
        src_vocab_size,
        src_pad_idx,
        forward_expansion,
        dropout: int=0,
        embed_dim: int=768,
        num_layers: int=6,
        num_heads: int=8,
        max_length: int=76,
        final_output_embed: int=256
    ):
        """
        The Final Transformer Module that goes through our Encoder block with a mask and outputs the word_embeddings of dimension 
        (final_output_embed)
        
        Arguments:-
        src_vocab_size: Total number of words in our Source Vocabulary
        src_pad_idx: The index designated for <PAD> token
        forward_expansion: Arbitrary expansion dimension (Check Transformer Block)
        embed_dim: Words are embedded into embed_size
        num_layers: Number of times to repeat transformer block
        num_heads: Number of heads to create for Multi-Head Attention layer
        #device: Deprecated for Lightning Implementation since lightning doesn't like device calls :D
        dropout: Dropout rate
        max_length: Maximum allowed sequence length, aids in getting positional embeddings
        final_output_embed: Final embedding dimension after passing through Transformer Block
        """
        super().__init__()
        
        self.encoder = Encoder(
            src_vocab_size,
            embed_dim,
            num_layers,
            num_heads,
            forward_expansion,
            dropout,
            max_length,
            final_output_embed
        )

        self.src_pad_idx = src_pad_idx
        
    def make_src_mask(self, src):
        src_mask = (src == self.src_pad_idx).unsqueeze(1).unsqueeze(2)  # True if src == src_pad_idx, else False
        return src_mask

    def forward(self, src):
        """
        Forward pass for the Transformer
        
        Arguments:-
        src: Source Input (num_of_samples, max_seq_length [Before CLS])
        """
        src = src.to(device=self.device)
        n = src.shape[0]
        tok = torch.full((n,1), 3).to(device=self.device)

        src = torch.cat((tok, src), 1) # Adding a CLS token to the sentences
        src_mask = self.make_src_mask(src) 
        enc_src = self.encoder(src, src_mask)      
        return enc_src

class VisualTransformer(pl.LightningModule):
    
    def __init__(
        self,
        chw,
        num_of_patches,
        forward_expansion: int=4,
        embed_dim: int=64,
        num_layers: int=6,
        num_heads: int=2,
        dropout: int=0,
        final_output_embed: int=256
    ):
        """
        The Visual Transformer patchifies our images into tokens and feeds them into a Transformer to get attentioned outputs
        
        Arguments:-
        chw: Tuple of Channels, Height, Width 
        num_of_patches: Number of patches to divide the height and width into
        forward_expansion: Arbitrary expansion dimension (Check Transformer Block)
        embed_dim: Words are embedded into embed_size
        num_layers: Number of times to repeat transformer block
        num_heads: Number of heads to create for Multi-Head Attention layer
        dropout: Dropout rate
        final_output_embed: Final embedding dimension after passing through Transformer Block
        """
        
        super().__init__()

        self.num_of_patches = num_of_patches
        self.chw = chw
        self.patch_size_h = self.chw[1] // num_of_patches
        self.patch_size_w = self.chw[2] // num_of_patches
        self.hidden_size = embed_dim  
        self.final_output_embed = final_output_embed 
        
        
        self.fc1 = nn.Linear(self.chw[0] * self.patch_size_h * self.patch_size_w, self.hidden_size)

        self.class_token = nn.Parameter(torch.rand(1, self.hidden_size))    
        self.pos_embed = nn.Parameter(torch.Tensor(get_positional_embeddings(self.num_of_patches ** 2 + 1, self.hidden_size))) 
        self.pos_embed.requires_grad = False
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_dim,
                    dropout,
                    num_heads,
                    forward_expansion
                )
                for _ in range(num_layers)
            ]
        )
        
        self.mlp = nn.Sequential(
                nn.Linear(self.hidden_size, self.final_output_embed)
            )
    
    def forward(
        self,
        image_inp
    ):

        img_width = image_inp.shape[-1]
        img_height = image_inp.shape[-2]
        
        assert img_width%self.num_of_patches == 0, 'Image width not divisible by number of patches'
        assert img_height%self.num_of_patches == 0, 'Image height not divisible by number of patches'

        patches = make_patches(image_inp, self.num_of_patches).to(device=self.device)
        tokens = self.fc1(patches)  
        tokens = torch.stack([torch.vstack((self.class_token, tokens[i])) for i in range(len(tokens))])
        
        positions = self.pos_embed.repeat(image_inp.shape[0], 1, 1)
        tokens += positions
        for layer in self.layers:
            out = layer(tokens, tokens, tokens, None) 

        out = self.mlp(out[:, 0])
        return out 

class CLIP(pl.LightningModule):
    
    def __init__(
        self,
        chw, 
        num_of_patches,
        src_vocab_size,
        device,
        src_pad_idx: int=0,
        forward_expansion: int=4
    ):
        super().__init__()
        self.visual_transformer = VisualTransformer(
            chw=chw, 
            num_of_patches=num_of_patches
        )
        self.text_transformer = Transformer(
            src_vocab_size,
            src_pad_idx,
            forward_expansion
        )
        self.maxie = 75
        print(f'We are now on {self.device}')
        
    
    def training_step(self, batch, batch_idx):
        img, cap = batch
        sents = [each_sent.split() for each_sent in cap]
        max_seq_length = self.maxie
        temperature = 0.07
        
        try:
            tokens = [[1] + [clipVocab.to_index(word) for word in sent] + [2] for sent in sents]
        except:
            print(f'Key error in tokenizer! Skipped batch number {batch_idx} :(')
            return None

        for idx, token in enumerate(tokens):
            if len(token) >= max_seq_length:
                token = token[:max_seq_length]
                token[-1] = 2
                tokens[idx] = torch.Tensor(token)
            if len(token) < max_seq_length:
                diff = max_seq_length - len(token)
                token = token + [0]*diff
                tokens[idx] = torch.Tensor(token)
        
        tokens = torch.vstack(tokens).to(torch.int64).to(device=self.device)
        img = img.to(device=self.device)
        
        I_f = self.visual_transformer(img)
        T_f = self.text_transformer(tokens)
        
        logits = ((T_f @ I_f.T) * np.exp(temperature))

        targets = torch.eye(logits.shape[0]).to(device=self.device)
        loss = F.cross_entropy(logits, targets)
        
        self.log('train_loss', loss, prog_bar=True)
        return loss
       
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr = 0.001)

In [76]:
class ImgCapDataModule(pl.LightningDataModule):
    
    def __init__(self, data_dir, batch_size: int = 64, num_workers: int = 4):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = 64
        self.num_workers = num_workers
        
    def setup(self, stage):
        self.preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor()
        ])
        
        self.train_ds = TextImageDataset(out_dir, self.preprocess)
        
    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=64,
            num_workers=self.num_workers,
            shuffle=False
        )

In [77]:
#Training config
chw = (3, 224, 224)
num_of_patches = 8
src_vocab_size = clipVocab.num_words
batch_size=64
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [72]:
clipModel = CLIP(chw=chw, num_of_patches=num_of_patches, src_vocab_size=src_vocab_size, device=device)

We are now on cpu


In [73]:
dm = ImgCapDataModule(data_dir="dataset/", batch_size=64)
trainer = pl.Trainer(accelerator='gpu', min_epochs=2, max_epochs=32)

In [74]:
trainer.fit(clipModel, dm)

Training: 0it [00:00, ?it/s]

Key error in tokenizer! Skipped batch number 47 :(
Key error in tokenizer! Skipped batch number 49 :(
Key error in tokenizer! Skipped batch number 64 :(
Key error in tokenizer! Skipped batch number 95 :(
Key error in tokenizer! Skipped batch number 98 :(
Key error in tokenizer! Skipped batch number 109 :(
Key error in tokenizer! Skipped batch number 132 :(
Key error in tokenizer! Skipped batch number 255 :(
Key error in tokenizer! Skipped batch number 258 :(
Key error in tokenizer! Skipped batch number 47 :(
Key error in tokenizer! Skipped batch number 49 :(
Key error in tokenizer! Skipped batch number 64 :(
Key error in tokenizer! Skipped batch number 95 :(
Key error in tokenizer! Skipped batch number 98 :(
Key error in tokenizer! Skipped batch number 109 :(
Key error in tokenizer! Skipped batch number 132 :(
